In [1]:
import torch

weight2D = torch.randint(0, 256, size=(8, 100))
uniqs, counts = torch.unique(weight2D, return_counts=True)

result = {}
subArray = 100
numSubArray = int(weight2D.shape[1]/subArray)
numSubRow = [subArray] * numSubArray + ([] if weight2D.shape[1] % subArray == 0 else [weight2D.shape[1] % subArray])
for s, rowArray in enumerate(numSubRow):
    mask = torch.zeros_like(weight2D)
    
    uniqs, counts = torch.unique(weight2D[:, (s*subArray):(s+1)*subArray], return_counts=True)
    result[s] = {'uniqs': uniqs, 'counts': counts}
    
print(result)








{0: {'uniqs': tensor([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
         14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,
         28,  29,  30,  31,  32,  33,  34,  36,  38,  39,  41,  42,  43,  44,
         45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,  56,  57,  58,
         59,  60,  61,  62,  64,  65,  66,  67,  68,  69,  70,  71,  72,  73,
         74,  75,  76,  77,  78,  79,  80,  81,  82,  83,  86,  87,  88,  89,
         90,  91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
        105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118,
        120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133,
        134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147,
        148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161,
        162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175,
        176, 177, 178, 179, 180, 181, 182, 183, 18

In [7]:
import torch
import torch.nn.functional as F

inputQ = torch.tensor([1,2,3,4,5,6]).reshape(2, 3).float()
weightQ = torch.tensor([1,2,3,4,5,6]).reshape(2, 3).float()
outputreal = F.linear(inputQ, weightQ, None)
print(outputreal)

def LinearQuantizeOut(x, k, alpha):                # only quantize >0 values (relu must be preceded)
    if k == 1:
        return torch.sign(x).add(1).div(2).mul(alpha)
    else:
        L = 2.**k - 1
        xdiv = x.div(alpha)
        xc = xdiv.clamp(min=0., max=1.)
        xq = xc.mul(L).round()
        xmul = xq.div(L).mul(alpha)
    return xmul

abits = 2
wbits = 2
adcbits = 2

outputShiftIN = torch.zeros_like(outputreal)
output_dist = []
print('dist: ', output_dist)
for z in range(abits):
    inputB = torch.fmod(inputQ, 2)              # 12,10,14 = [0,0,0] / [0,1,1] / [1,0,1] / [1,1,1]
    inputQ = torch.round((inputQ-inputB)/2)     # 12,10,14 = [6,5,7] / [3,2,3] / [1,1,1] / [0,0,0]
    weightQb = weightQ
    outputShiftW = torch.zeros_like(outputreal)
    for k in range(wbits):
        weightB = torch.fmod(weightQb, 2)
        weightQb = torch.round((weightQb-weightB)/2)
        threshold = int(weightQ.sum() / subArray * 1.0)
        
        outputPartial = F.linear(inputB, weightB, None)
        print(outputPartial)
        output_dist.append(outputPartial)
        outputPartial = torch.clamp(outputPartial, 0, 3)
        outputADC = LinearQuantizeOut(outputPartial, adcbits, 2)
        # shift per w bit sequence
        outputShiftW = outputShiftW + outputADC * (2 ** k)
    # shift per input bit sequence
    outputShiftIN = outputShiftIN + outputShiftW * (2 ** z)
print(outputShiftIN)
output_dist = torch.stack(output_dist)
print(output_dist)
out_mean = output_dist.mean()
out_var = output_dist.var()
quant_err = (outputShiftIN - outputreal).abs().sum().div(outputADC.nelement()).item()
# since inputQ [0, 15] when k=4, rescale output by divide 16
# output = output + inputS * (outputIN * wS + outputIND * w.min())     # suppose I=[0~15], W=[-8~7] -> I*W = I[0~15]*W[0~15] + I[0~15]*W_constant[-8] (which is w.min())

print(out_mean)
print(out_var)

tensor([[14., 32.],
        [32., 77.]])
dist:  []
tensor([[2., 0.],
        [0., 1.]])
tensor([[1., 1.],
        [1., 0.]])
tensor([[1., 1.],
        [1., 0.]])
tensor([[2., 1.],
        [1., 1.]])
tensor([[15.3333, 10.6667],
        [10.6667,  6.6667]])
tensor([[[2., 0.],
         [0., 1.]],

        [[1., 1.],
         [1., 0.]],

        [[1., 1.],
         [1., 0.]],

        [[2., 1.],
         [1., 1.]]])
tensor(0.8750)
tensor(0.3833)


In [16]:
import torch
import torch.nn.functional as F

w = torch.load("/app/weightQ.pth", map_location='cpu')
x = torch.arange(0, w.flatten().size(0), step=1).float().reshape(-1, w.shape[1])

print(x.size())
print(w.shape)
print(w)

out = F.linear(x, w, None)
print(out.size())





torch.Size([16, 144])
torch.Size([16, 144])
tensor([[149., 153., 143.,  ..., 153., 128., 150.],
        [142., 143., 146.,  ..., 145., 143., 145.],
        [153., 155., 152.,  ..., 157., 152., 166.],
        ...,
        [154., 159., 158.,  ..., 157., 162., 158.],
        [145., 135., 146.,  ..., 170., 161., 127.],
        [149., 149., 159.,  ..., 102., 111., 145.]])
torch.Size([16, 16])
